## **Custom CNN**

This notebook documents the creation and training of the custom Convolutional Neural Network (CNN). This serves as the baseline model against which the performance of the transfer learning models will be compared.

Goal: Train a Custom CNN using 80/10/10 stratified split, 0-1 normalization, data augmentation, and class weights (due to class imbalance).

## Project Setup and Initialization

Imports and Paths

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
import os

In [ ]:
!pip install data_utils

ERROR: Ignored the following versions that require a different python version: 0.0.1 Requires-Python >=2.6,<=3.0; 0.0.2 Requires-Python >=2.6,<=3.0; 0.0.3 Requires-Python >=2.6,<=3.0; 0.0.4 Requires-Python >=2.6,<=3.0; 0.0.5 Requires-Python >=2.6,<=3.0; 0.0.6 Requires-Python >=2.6,<=3.0; 0.0.7 Requires-Python >=2.6,<=3.0
ERROR: Could not find a version that satisfies the requirement data_utils (from versions: none)
ERROR: No matching distribution found for data_utils


In [ ]:
# Import utility functions from  uploaded files
from data_utils import perform_stratified_split, DataGeneratorUtils, TARGET_SIZE, SEED
from train_utils import compile_model, get_callbacks

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Path to your metadata.csv file
METADATA_PATH = "/content/drive/MyDrive/Deep_Learning/rare_species/metadata.csv"
# Root directory containing all your original, un-split image files
IMAGE_ROOT_DIR = "/content/drive/MyDrive/Deep_Learning/rare_species"
# The target folder where the stratified data structure (train/val/test) will be created
DATA_TARGET_DIR = "/content/drive/MyDrive/Deep_Learning/data"

# Create the results directory if it doesn't exist to save model weights
os.makedirs("/content/drive/MyDrive/Deep_Learning/outputs", exist_ok=True)

Data Splitting and Generator Creation

*We use the functions from data_utils.py to handle the reproducible split and the data pipeline creation.*

In [ ]:
# Load the metadata file
try:
    metadata_df = pd.read_csv(METADATA_PATH)
except FileNotFoundError:
    print("ERROR: Metadata file not found.")

In [ ]:
# IMPORTANT: RUN perform_stratified_split ONLY ONCE!
data_base_path = perform_stratified_split(metadata_df, IMAGE_ROOT_DIR, DATA_TARGET_DIR)
print(f"Data structure created/verified at: {data_base_path}")

Data Split: Train=9585, Validation=1199, Test=1199
Organizing train set...
Organizing validation set...
Organizing test set...
Data directory structure successfully created/updated.
Data structure created/verified at: /content/drive/MyDrive/Deep_Learning/data


In [ ]:
#!ls -R /content/drive


note: there is a typo in the path file - rive instead of drive

Verification: counting images per class in split directories

In [ ]:
def count_images_per_class(base_directory, set_name):
    """Counts the number of images in each class (family folder) for a given set."""
    directory = os.path.join(base_directory, set_name)
    class_counts = {}

    if not os.path.exists(directory):
        return {f"ERROR: Directory not found at {directory}": 0}

    for folder in os.listdir(directory):
        folder_path = os.path.join(directory, folder)
        if os.path.isdir(folder_path):
            # Count files in the class folder
            image_count = len([f for f in os.listdir(folder_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))])
            class_counts[folder] = image_count
    return class_counts

In [ ]:
# Count images in train, validation, and test directories
train_class_counts = count_images_per_class(DATA_TARGET_DIR, 'train')
val_class_counts = count_images_per_class(DATA_TARGET_DIR, 'validation')
test_class_counts = count_images_per_class(DATA_TARGET_DIR, 'test')

# Display results
print("\n--- Class Counts Verification ---")
print("Number of images per class in the TRAIN directory (Top 5):")
# Sort and print for readability
print(pd.Series(train_class_counts).sort_values(ascending=False).head(5))

print("\nNumber of images per class in the VALIDATION directory (Top 5):")
print(pd.Series(val_class_counts).sort_values(ascending=False).head(5))

print("\nNumber of images per class in the TEST directory (Top 5):")
print(pd.Series(test_class_counts).sort_values(ascending=False).head(5))

# Check for overall size consistency
total_counted = sum(train_class_counts.values()) + sum(val_class_counts.values()) + sum(test_class_counts.values())
print(f"\nTotal images successfully counted across all splits: {total_counted}")


--- Class Counts Verification ---
Number of images per class in the TRAIN directory (Top 5):
cercopithecidae    240
dactyloidae        240
formicidae         233
plethodontidae     216
carcharhinidae     216
dtype: int64

Number of images per class in the VALIDATION directory (Top 5):
cercopithecidae    30
dactyloidae        30
formicidae         29
plethodontidae     27
carcharhinidae     27
dtype: int64

Number of images per class in the TEST directory (Top 5):
dactyloidae        30
cercopithecidae    30
formicidae         29
plethodontidae     27
salamandridae      27
dtype: int64

Total images successfully counted across all splits: 11983


In [ ]:
data_base_path = DATA_TARGET_DIR


In [ ]:
#  Initialize Data Generators
data_util = DataGeneratorUtils(data_base_path)

train_generator = data_util.create_generators('train')
val_generator = data_util.create_generators('validation')


Found 9585 images belonging to 202 classes.
Found 1199 images belonging to 202 classes.


In [ ]:
NUM_CLASSES = train_generator.num_classes

In [ ]:
# Calculate Class Weights (Crucial for rare species imbalance)
class_weights = data_util.calculate_class_weights(train_generator)

print(f"\nSetup complete. Total classes: {NUM_CLASSES}")
print(f"Train samples: {train_generator.samples}, Validation samples: {val_generator.samples}")

Class weights calculated for 202 classes.

Setup complete. Total classes: 202
Train samples: 9585, Validation samples: 1199


In [ ]:
# --- INSERT THIS AT THE START OF YOUR CELL ---
import os
import pandas as pd
from data_utils import perform_stratified_split

# 1. Check if data exists. If not, recreate it.
DATA_DIR = 'data'
if not os.path.exists(os.path.join(DATA_DIR, 'train')):
    print(f"⚠️ Data folder not found (Runtime restarted). Re-generating split...")

    # Define Source Paths (Drive)
    METADATA_PATH = "/content/drive/MyDrive/Deep_Learning/rare_species/metadata.csv"
    IMAGE_ROOT = "/content/drive/MyDrive/Deep_Learning/rare_species/"

    try:
        df = pd.read_csv(METADATA_PATH)
        # This copies files from Drive -> Local Colab (Takes ~10-20 mins)
        perform_stratified_split(df, IMAGE_ROOT, data_target_dir=DATA_DIR)
        print("✅ Data successfully restored to local environment.")
    except Exception as e:
        print(f"❌ Error restoring data: {e}")
        raise
else:
    print("✅ Data folder found. Skipping split.")
# ---------------------------------------------

⚠️ Data folder not found (Runtime restarted). Re-generating split...
Data Split: Train=9585, Validation=1199, Test=1199
Organizing train set...
Organizing validation set...
Organizing test set...
Data directory structure successfully created/updated.
✅ Data successfully restored to local environment.


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetV2B0
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
import os
import shutil
import numpy as np

# --- 1. CONFIGURATION ---
# Define paths (Adjust 'data' if you are pulling from Drive)
DATA_DIR = 'data'
if not os.path.exists(DATA_DIR):
    # Fallback to Drive if local data was wiped
    DATA_DIR = "/content/drive/MyDrive/Deep_Learning/organized_data_224"

DRIVE_SAVE_PATH = "/content/drive/MyDrive/Deep_Learning/models/"
os.makedirs(DRIVE_SAVE_PATH, exist_ok=True)

# --- 2. THE FIX: GENERATORS WITHOUT RESCALING ---
# EfficientNetV2 expects pixels in [0, 255], NOT [0, 1]
class DataGeneratorUtils_Fixed:
    def __init__(self, data_root):
        self.data_root = data_root

    def _custom_preprocess(self, image):
        if tf.shape(image)[-1] == 1:
            image = tf.image.grayscale_to_rgb(image)
        return image

    def create_generators(self, set_type):
        data_dir = os.path.join(self.data_root, set_type)
        if set_type == 'train':
            datagen = ImageDataGenerator(
                preprocessing_function=self._custom_preprocess, # No rescale!
                rotation_range=20,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                horizontal_flip=True,
                fill_mode='nearest'
            )
        else:
            datagen = ImageDataGenerator(
                preprocessing_function=self._custom_preprocess # No rescale!
            )

        return datagen.flow_from_directory(
            data_dir,
            target_size=(224, 224),
            batch_size=32,
            class_mode='categorical',
            shuffle=(set_type == 'train'),
            seed=42
        )

print(f"Loading Fixed Generators from: {DATA_DIR}")
utils = DataGeneratorUtils_Fixed(DATA_DIR)
train_generator = utils.create_generators('train')
val_generator = utils.create_generators('validation')

# Calculate weights again to be safe
from sklearn.utils import class_weight
y_train = train_generator.classes
class_weights_arr = class_weight.compute_class_weight(
    class_weight='balanced', classes=np.unique(y_train), y=y_train
)
class_weights = dict(enumerate(class_weights_arr))
num_classes = len(train_generator.class_indices)

# --- 3. MODEL DEFINITION ---
def build_efficientnet_model(input_shape, num_classes):
    base_model = EfficientNetV2B0(
        weights='imagenet',
        include_top=False,
        input_shape=input_shape
    )
    base_model.trainable = False  # Start Frozen

    model = models.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.BatchNormalization(),
        layers.Dropout(0.3),
        layers.Dense(512, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.4),
        layers.Dense(num_classes, activation='softmax')
    ])
    return base_model, model

def get_callbacks(phase_name):
    os.makedirs('results', exist_ok=True)
    return [
        ModelCheckpoint(
            filepath=f'results/best_{phase_name}.weights.h5',
            monitor='val_loss', save_best_only=True, save_weights_only=True, verbose=1
        ),
        EarlyStopping(monitor='val_loss', patience=6, restore_best_weights=True, verbose=1),
        ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, verbose=1)
    ]

# Build Model
base_model, model = build_efficientnet_model((224, 224, 3), num_classes)

# ==========================================
# PHASE 1: TRAIN HEAD (FROZEN BASE)
# ==========================================
print(f"\n🔥 STARTING PHASE 1: Head Training (Frozen)...")

model.compile(
    optimizer=optimizers.Adam(learning_rate=0.001),
    loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.1),
    metrics=['accuracy']
)

history_p1 = model.fit(
    train_generator,
    epochs=12,
    validation_data=val_generator,
    callbacks=get_callbacks('effnet_phase1'),
    class_weight=class_weights
)

# Save Phase 1
model.save('efficientnet_phase1.keras')
shutil.copy('efficientnet_phase1.keras', os.path.join(DRIVE_SAVE_PATH, 'efficientnet_phase1.keras'))
print("✅ Phase 1 Complete. Saved.")

# ==========================================
# PHASE 2: FINE-TUNING (UNFROZEN BASE)
# ==========================================
print(f"\n🔥 STARTING PHASE 2: Fine-Tuning (Unfrozen)...")

# 1. Unfreeze the base
base_model.trainable = True

# 2. Keep the bottom layers frozen (Standard practice to preserve edge detection)
# EfficientNetV2B0 has ~270 layers. Let's freeze the bottom 200.
fine_tune_at = 200
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

print(f"Number of layers in base model: {len(base_model.layers)}")
print(f"Training layers starting from layer {fine_tune_at}")

# 3. Re-compile with LOW Learning Rate (Critical!)
model.compile(
    optimizer=optimizers.Adam(learning_rate=1e-5), # 100x smaller LR
    loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.1),
    metrics=['accuracy']
)

# 4. Train again
history_p2 = model.fit(
    train_generator,
    epochs=20,
    validation_data=val_generator,
    callbacks=get_callbacks('effnet_phase2_finetune'),
    class_weight=class_weights
)

# --- FINAL SAVE ---
final_name = 'efficientnet_final_finetuned.keras'
model.save(final_name)
shutil.copy(final_name, os.path.join(DRIVE_SAVE_PATH, final_name))
print(f"🎉 ALL DONE! Final model saved to: {os.path.join(DRIVE_SAVE_PATH, final_name)}")

Loading Fixed Generators from: data
Found 9585 images belonging to 202 classes.
Found 1199 images belonging to 202 classes.

🔥 STARTING PHASE 1: Head Training (Frozen)...


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/12
300/300 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.1496 - loss: 4.7627

/usr/local/lib/python3.12/dist-packages/PIL/Image.py:3452: DecompressionBombWarning: Image size (115600000 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(



Epoch 1: val_loss improved from inf to 2.51617, saving model to results/best_effnet_phase1.weights.h5
300/300 ━━━━━━━━━━━━━━━━━━━━ 435s 1s/step - accuracy: 0.1499 - loss: 4.7601 - val_accuracy: 0.5296 - val_loss: 2.5162 - learning_rate: 0.0010
Epoch 2/12
300/300 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.4513 - loss: 2.7027
Epoch 2: val_loss improved from 2.51617 to 2.31497, saving model to results/best_effnet_phase1.weights.h5
300/300 ━━━━━━━━━━━━━━━━━━━━ 383s 1s/step - accuracy: 0.4513 - loss: 2.7027 - val_accuracy: 0.5788 - val_loss: 2.3150 - learning_rate: 0.0010
Epoch 3/12
300/300 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.5308 - loss: 2.3688
Epoch 3: val_loss improved from 2.31497 to 2.20218, saving model to results/best_effnet_phase1.weights.h5
300/300 ━━━━━━━━━━━━━━━━━━━━ 389s 1s/step - accuracy: 0.5308 - loss: 2.3689 - val_accuracy: 0.6314 - val_loss: 2.2022 - learning_rate: 0.0010
Epoch 4/12
300/300 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.5723 - loss: 2.1989
Epoc

# Hyperparameter Tuner and Callbacks

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

In [ ]:
PROJECT_NAME = 'custom_cnn_hyperband'

In [ ]:
# Early Stopping: Stops training if val_loss doesn't improve for 5 epochs
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True,
    verbose=1
)

In [ ]:
# Reduce LR on Plateau: Reduces LR if val_loss doesn't improve for 5 epochs
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.2,
    patience=5,
    min_lr=1e-6,
    verbose=1
)

In [ ]:
callbacks = [early_stopping, reduce_lr]

In [ ]:
#Instantiate the Hyperband Tuner
tuner = kt.Hyperband(
    hypermodel=build_hypermodel,
    objective='val_accuracy', # Maximize validation accuracy
    max_epochs=30,           # Max epochs for a full training run
    factor=3,                # Halving factor for Hyperband
    directory='/content/drive/MyDrive/NOVA_IMS/Deep_Learning_Project/outputs', # Path to save results
    project_name=PROJECT_NAME,
    overwrite=True           # Overwrite previous search results
)

print("Tuner search space summary:")
tuner.search_space_summary()

In [ ]:
# Run the search using the training and validation generators and class weights
tuner.search(
    train_generator,
    epochs=30,
    validation_data=val_generator,
    callbacks=callbacks,
    class_weight=class_weights,
    verbose=1
)